In [1]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

project_id = 'crazy-coffee-jam'

from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client(project=project_id)

## Check sequence user drop by inter level 8

In [14]:
query = f"""
SELECT
  user_pseudo_id,
  event_name,
  event_timestamp,
  ev.key,
  COALESCE(ev.value.string_value,
           CAST(ev.value.int_value AS STRING),
           CAST(ev.value.float_value AS STRING),
           CAST(ev.value.double_value AS STRING)
  ) AS value
FROM
  `crazy-coffee-jam.analytics_483028617.events_intraday_*`,
  UNNEST(event_params) AS ev
WHERE
  _TABLE_SUFFIX BETWEEN '20250422' AND '20250519'
  and user_pseudo_id in ('{("','".join(drop_after_inter_users))}')
  and key not in ('firebase_event_origin','firebase_screen_class','ga_session_number', 'engaged_session_event', 'firebase_screen_id', 'ga_session_id', 'firebase_event_id',
  'engagement_time_msec', 'system_app', 'previous_first_open_count', 'system_app_update', 'firebase_conversion', 'update_with_analytics', 'level_percent','IsLoop',
  'value', 'ad_platform', 'ad_unit_name', 'currency','ad_source')
ORDER BY
  user_pseudo_id,
  event_timestamp
"""
sequence_user = client.query(query).to_dataframe()

In [15]:
sequence_user

,user_pseudo_id,event_name,event_timestamp,key,value
0,001a9c6f17af62e29f81d897699c7682,screen_view,1747259109613089,entrances,1
1,001a9c6f17af62e29f81d897699c7682,firebase_campaign,1747259109983088,campaign_info_source,referrer API v2
2,001a9c6f17af62e29f81d897699c7682,firebase_campaign,1747259109983088,medium,(not set)
3,001a9c6f17af62e29f81d897699c7682,firebase_campaign,1747259109983088,source,(not set)
4,001a9c6f17af62e29f81d897699c7682,start_level,1747259117918091,level,1
...,...,...,...,...,...
153684,fff7b258d6cddb69af658de0b448b768,resource_gain,1747202533623263,resource_amount,10
153685,fff7b258d6cddb69af658de0b448b768,screen_view,1747202541288265,firebase_previous_id,-2488493241754808185
153686,fff7b258d6cddb69af658de0b448b768,screen_view,1747202541288265,firebase_previous_class,MessagingUnityPlayerActivity
153687,fff7b258d6cddb69af658de0b448b768,ad_impression,1747202542502252,ad_format,INTER


In [16]:
# Chỉ lấy các event sau khi user bắt đầu level 8
# Get the timestamp when each user started level 8
level_8_start_times = sequence_user[
    (sequence_user['event_name'] == 'start_level') &
    (sequence_user['key'] == 'level') &
    (sequence_user['value'] == '8')
].groupby('user_pseudo_id')['event_timestamp'].min()

# Filter sequence_user to only include events after each user's level 8 start
sequence_user = sequence_user.merge(
    level_8_start_times.reset_index(),
    on='user_pseudo_id',
    suffixes=('', '_level8_start')
)
sequence_user = sequence_user[
    sequence_user['event_timestamp'] >= sequence_user['event_timestamp_level8_start']
].drop('event_timestamp_level8_start', axis=1)

# Bỏ các dòng có event_name là booster_use và key là level và value là 8
sequence_user = sequence_user[
    ~((sequence_user['event_name'] == 'booster_use') &
      (sequence_user['key'] == 'level') &
      (sequence_user['value'] == '8'))
]

# Drop rows where value is 'BANNER'
sequence_user = sequence_user[sequence_user['value'] != 'BANNER']
sequence_user = sequence_user[sequence_user['value'] != 'LEADER']

# For ad_impression events, replace event_name with the corresponding value
sequence_user.loc[sequence_user['event_name'] == 'ad_impression', 'event_name'] = sequence_user.loc[sequence_user['event_name'] == 'ad_impression', 'value']

sequence_user

,user_pseudo_id,event_name,event_timestamp,key,value
125,001a9c6f17af62e29f81d897699c7682,start_level,1747259435318251,level,8
126,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,level,8
127,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_type,Frezze
128,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_method,Free
129,001a9c6f17af62e29f81d897699c7682,booster_use,1747259439858253,booster_type,Frezze
...,...,...,...,...,...
153684,fff7b258d6cddb69af658de0b448b768,resource_gain,1747202533623263,resource_amount,10
153685,fff7b258d6cddb69af658de0b448b768,screen_view,1747202541288265,firebase_previous_id,-2488493241754808185
153686,fff7b258d6cddb69af658de0b448b768,screen_view,1747202541288265,firebase_previous_class,MessagingUnityPlayerActivity
153687,fff7b258d6cddb69af658de0b448b768,INTER,1747202542502252,ad_format,INTER


In [18]:
# Count number of rows for each level value where key = 'level'
level_counts = sequence_user[sequence_user['key'] == 'level'].groupby('value').size().reset_index(name='count')
print("Level counts:")
print(level_counts)

Level counts:
  value  count
0    01      8
1     6      9
2     7      2
3     8  11039
4     9    645


In [19]:
print("Rows with key='level' and value != '8':")
print(sequence_user[(sequence_user['key'] == 'level') & (sequence_user['value'] != '8')])

Rows with key='level' and value != '8':
                          user_pseudo_id     event_name   event_timestamp  \
171     001a9c6f17af62e29f81d897699c7682  resource_gain  1747259542054282   
387     002d60067763bd3fb22ffd24ad1dd424  resource_gain  1747346674205913   
621     01016693579df9c72c3c0b94fcb9609c  resource_gain  1747072524648806   
846     01a131f975505e3eec9ff4f499cd3826  resource_gain  1746648693573755   
1079    01c5688a99cd703075ac8847025d2ae3  resource_gain  1746729932428475   
...                                  ...            ...               ...   
152787  ff73128446a78f62a5e088fd76236069  resource_gain  1747507230338784   
153018  ff78eeca154db2b41a8a39eacc9f4ad2  resource_gain  1746420571387366   
153224  ff8ea320a832ac7f4f52e98a2660bd7a  resource_gain  1746294256059304   
153427  ff9785c48aa0a3cb80f06202a5acdd0d  resource_gain  1747480718226752   
153681  fff7b258d6cddb69af658de0b448b768  resource_gain  1747202533623263   

          key value  
171     level

In [20]:
print("Number of users who reached level 9:")
print(sequence_user[
    (sequence_user['key'] == 'level') & 
    (sequence_user['value'] == '9')
]['user_pseudo_id'].nunique())

Number of users who reached level 9:
642


In [21]:
level9 = sequence_user[(sequence_user['key'] == 'level') & (sequence_user['value'] == '9')]
level9['event_name'].unique()

array(['resource_gain'], dtype=object)

In [23]:
df = sequence_user.sort_values(['user_pseudo_id', 'event_timestamp'], ascending=[True, True])

In [25]:
df[df['user_pseudo_id'] == '001a9c6f17af62e29f81d897699c7682']

,user_pseudo_id,event_name,event_timestamp,key,value
125,001a9c6f17af62e29f81d897699c7682,start_level,1747259435318251,level,8
126,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,level,8
127,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_type,Frezze
128,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_method,Free
129,001a9c6f17af62e29f81d897699c7682,booster_use,1747259439858253,booster_type,Frezze
133,001a9c6f17af62e29f81d897699c7682,box_complete,1747259454186256,level,8
134,001a9c6f17af62e29f81d897699c7682,box_complete,1747259454186256,box_id,5
135,001a9c6f17af62e29f81d897699c7682,box_complete,1747259459632257,box_id,1
136,001a9c6f17af62e29f81d897699c7682,box_complete,1747259459632257,level,8
139,001a9c6f17af62e29f81d897699c7682,box_complete,1747259464417260,level,8


In [28]:
query = f"""
SELECT
  user_pseudo_id,
  event_name,
  event_timestamp,
  ev.key,
  COALESCE(ev.value.string_value,
           CAST(ev.value.int_value AS STRING),
           CAST(ev.value.float_value AS STRING),
           CAST(ev.value.double_value AS STRING)
  ) AS value
FROM
  `crazy-coffee-jam.analytics_483028617.events_intraday_*`,
  UNNEST(event_params) AS ev
WHERE
  _TABLE_SUFFIX BETWEEN '20250422' AND '20250519'
  and key  in ('level')
  and event_name in ('win_level', 'resource_gain')
  and user_pseudo_id in (select distinct user_pseudo_id from `crazy-coffee-jam.flatten_table.win_level` where event_date between '2025-05-10' and '2025-05-12' and level between 10 and 15)
ORDER BY
  user_pseudo_id,
  event_timestamp
"""
sequence_user = client.query(query).to_dataframe()
sequence_user

,user_pseudo_id,event_name,event_timestamp,key,value
0,001b3ded69198cfa4b187c7640ce78d6,win_level,1746738567942991,level,1
1,001b3ded69198cfa4b187c7640ce78d6,resource_gain,1746738567943992,level,2
2,001b3ded69198cfa4b187c7640ce78d6,win_level,1746738577551000,level,2
3,001b3ded69198cfa4b187c7640ce78d6,resource_gain,1746738577552001,level,3
4,001b3ded69198cfa4b187c7640ce78d6,win_level,1746738588234008,level,3
...,...,...,...,...,...
142479,ffe28580ce67ac3e3c8ed05f6cff80db,win_level,1747497951870185,level,36
142480,ffe28580ce67ac3e3c8ed05f6cff80db,resource_gain,1747497951873186,level,37
142481,ffe28580ce67ac3e3c8ed05f6cff80db,resource_gain,1747671406466974,level,37
142482,ffe28580ce67ac3e3c8ed05f6cff80db,win_level,1747671812808125,level,37


In [29]:
sequence_user.sort_values(['user_pseudo_id', 'event_timestamp'], ascending=[True, True])

,user_pseudo_id,event_name,event_timestamp,key,value
0,001b3ded69198cfa4b187c7640ce78d6,win_level,1746738567942991,level,1
1,001b3ded69198cfa4b187c7640ce78d6,resource_gain,1746738567943992,level,2
2,001b3ded69198cfa4b187c7640ce78d6,win_level,1746738577551000,level,2
3,001b3ded69198cfa4b187c7640ce78d6,resource_gain,1746738577552001,level,3
4,001b3ded69198cfa4b187c7640ce78d6,win_level,1746738588234008,level,3
...,...,...,...,...,...
142479,ffe28580ce67ac3e3c8ed05f6cff80db,win_level,1747497951870185,level,36
142480,ffe28580ce67ac3e3c8ed05f6cff80db,resource_gain,1747497951873186,level,37
142481,ffe28580ce67ac3e3c8ed05f6cff80db,resource_gain,1747671406466974,level,37
142482,ffe28580ce67ac3e3c8ed05f6cff80db,win_level,1747671812808125,level,37


# Sequence of user drop level 8

In [2]:
query = """
with a as (
select distinct user_pseudo_id
from `crazy-coffee-jam.flatten_table.start_level`
where level >= 9
and user_pseudo_id in (select distinct user_pseudo_id from `crazy-coffee-jam.flatten_table.first_open` where event_date >= '2025-05-01')
)
select distinct user_pseudo_id
from `crazy-coffee-jam.flatten_table.start_level`
where event_date >= '2025-05-01'
and level = 8
and user_pseudo_id not in (select user_pseudo_id from a)
and user_pseudo_id in (select distinct user_pseudo_id from `crazy-coffee-jam.flatten_table.first_open` where event_date >= '2025-05-01')
"""
user_drop_level_8 = client.query(query).to_dataframe()
user_drop_level_8_list = user_drop_level_8['user_pseudo_id'].tolist()

In [3]:
query = f"""
SELECT
  user_pseudo_id,
  event_name,
  event_timestamp,
  ev.key,
  COALESCE(ev.value.string_value,
           CAST(ev.value.int_value AS STRING),
           CAST(ev.value.float_value AS STRING),
           CAST(ev.value.double_value AS STRING)
  ) AS value
FROM
  `crazy-coffee-jam.analytics_483028617.events_intraday_*`,
  UNNEST(event_params) AS ev
WHERE
  _TABLE_SUFFIX BETWEEN '20250501' AND '20250525'
  and user_pseudo_id in ('{("','".join(user_drop_level_8_list))}')
  and event_name in ('start_level', 'win_level', 'booster_use','ad_impression','box_complete')
  and key in ('level','ad_format')
ORDER BY
  user_pseudo_id,   
  event_timestamp
"""
sequence_user = client.query(query).to_dataframe()

In [4]:
sequence_user

,user_pseudo_id,event_name,event_timestamp,key,value
0,0005abff796753973e9ef6241e02d636,ad_impression,1746450163534886,ad_format,BANNER
1,0005abff796753973e9ef6241e02d636,start_level,1746450164459890,level,1
2,0005abff796753973e9ef6241e02d636,box_complete,1746450171810700,level,1
3,0005abff796753973e9ef6241e02d636,box_complete,1746450173927703,level,1
4,0005abff796753973e9ef6241e02d636,win_level,1746450175884704,level,1
...,...,...,...,...,...
165623,fff7b258d6cddb69af658de0b448b768,ad_impression,1747202520923251,ad_format,BANNER
165624,fff7b258d6cddb69af658de0b448b768,box_complete,1747202527616258,level,8
165625,fff7b258d6cddb69af658de0b448b768,box_complete,1747202531617261,level,8
165626,fff7b258d6cddb69af658de0b448b768,win_level,1747202533570262,level,8


In [5]:
# Chỉ lấy các event sau khi user bắt đầu level 8
# Get the timestamp when each user started level 8
level_8_start_times = sequence_user[
    (sequence_user['event_name'] == 'start_level') &
    (sequence_user['key'] == 'level') &
    (sequence_user['value'] == '8')
].groupby('user_pseudo_id')['event_timestamp'].min()

# Filter sequence_user to only include events after each user's level 8 start
sequence_user = sequence_user.merge(
    level_8_start_times.reset_index(),
    on='user_pseudo_id',
    suffixes=('', '_level8_start')
)
sequence_user = sequence_user[
    sequence_user['event_timestamp'] >= sequence_user['event_timestamp_level8_start']
].drop('event_timestamp_level8_start', axis=1)

# # Bỏ các dòng có event_name là booster_use và key là level và value là 8
# sequence_user = sequence_user[
#     ~((sequence_user['event_name'] == 'booster_use') &
#       (sequence_user['key'] == 'level') &
#       (sequence_user['value'] == '8'))
# ]

In [6]:
# Count number of rows for each level value where key = 'level'
level_counts = sequence_user[sequence_user['key'] == 'level'].groupby('value').size().reset_index(name='count')
print("Level counts:")
print(level_counts)

Level counts:
  value  count
0     1     17
1    10      7
2     2      8
3     3     10
4     4     12
5     5     16
6     6     26
7     7      8
8     8  22354


In [7]:
# prompt: sequence_user: Bỏ đi các dòng có key = level và value ứng với level đó != 8

sequence_user = sequence_user[
    ~((sequence_user['key'] == 'level') &
      (pd.to_numeric(sequence_user['value'], errors='coerce') != 8))
]

In [118]:
# Count number of rows for each level value where key = 'level'
level_counts = sequence_user[sequence_user['key'] == 'level'].groupby('value').size().reset_index(name='count')
print("Level counts:")
print(level_counts)

Level counts:
  value  count
0     8  16579


In [119]:
sequence_user

,user_pseudo_id,event_name,event_timestamp,key,value
71,0005abff796753973e9ef6241e02d636,start_level,1746524668723519,level,8
72,0005abff796753973e9ef6241e02d636,booster_use,1746524674356521,level,8
73,0005abff796753973e9ef6241e02d636,box_complete,1746524746387522,level,8
74,0005abff796753973e9ef6241e02d636,box_complete,1746524789730523,level,8
75,0005abff796753973e9ef6241e02d636,ad_impression,1746524796636517,ad_format,BANNER
...,...,...,...,...,...
124860,fff7b258d6cddb69af658de0b448b768,ad_impression,1747202520923251,ad_format,BANNER
124861,fff7b258d6cddb69af658de0b448b768,box_complete,1747202527616258,level,8
124862,fff7b258d6cddb69af658de0b448b768,box_complete,1747202531617261,level,8
124863,fff7b258d6cddb69af658de0b448b768,win_level,1747202533570262,level,8


In [8]:
# Drop rows where value is 'BANNER'
sequence_user = sequence_user[sequence_user['value'] != 'BANNER']
sequence_user = sequence_user[sequence_user['value'] != 'LEADER']
sequence_user = sequence_user[sequence_user['value'] != 'REWARDED']

# For ad_impression events, replace event_name with the corresponding value
sequence_user.loc[sequence_user['event_name'] == 'ad_impression', 'event_name'] = sequence_user.loc[sequence_user['event_name'] == 'ad_impression', 'value']

sequence_user

,user_pseudo_id,event_name,event_timestamp,key,value
71,0005abff796753973e9ef6241e02d636,start_level,1746524668723519,level,8
72,0005abff796753973e9ef6241e02d636,booster_use,1746524674356521,level,8
73,0005abff796753973e9ef6241e02d636,box_complete,1746524746387522,level,8
74,0005abff796753973e9ef6241e02d636,box_complete,1746524789730523,level,8
76,0005abff796753973e9ef6241e02d636,box_complete,1746524802171527,level,8
...,...,...,...,...,...
165622,fff7b258d6cddb69af658de0b448b768,box_complete,1747202518242255,level,8
165624,fff7b258d6cddb69af658de0b448b768,box_complete,1747202527616258,level,8
165625,fff7b258d6cddb69af658de0b448b768,box_complete,1747202531617261,level,8
165626,fff7b258d6cddb69af658de0b448b768,win_level,1747202533570262,level,8


In [9]:
# Group events by user_pseudo_id and aggregate event_names into a sequence
sequence_df = sequence_user.groupby('user_pseudo_id').agg(
    sequence=('event_name', lambda x: ' -> '.join(x))
).reset_index()

sequence_df

,user_pseudo_id,sequence
0,0005abff796753973e9ef6241e02d636,start_level -> booster_use -> box_complete -> ...
1,000b1cf405ea9ac1ae37ac761815cba7,start_level -> booster_use -> start_level
2,002d60067763bd3fb22ffd24ad1dd424,start_level -> booster_use -> box_complete -> ...
3,002f795faeb69377468c27da3364e743,start_level -> booster_use -> box_complete -> ...
4,008c00273819a7ccbcac538c1b687041,start_level -> booster_use -> box_complete
...,...,...
2223,ff8ea320a832ac7f4f52e98a2660bd7a,start_level -> booster_use -> box_complete -> ...
2224,ff934ad018e44b6ce2bfd79e7c109724,start_level -> booster_use -> box_complete -> ...
2225,ff9785c48aa0a3cb80f06202a5acdd0d,start_level -> booster_use -> box_complete -> ...
2226,ffd9c30e04c487cee4701bb35a060749,start_level -> booster_use -> booster_use


In [10]:
# Replace multiple consecutive box_complete events with a single box_complete
sequence_df['sequence'] = sequence_df['sequence'].str.replace(r'(box_complete(?:\s*->\s*box_complete)*)', 'box_complete', regex=True)
sequence_df

,user_pseudo_id,sequence
0,0005abff796753973e9ef6241e02d636,start_level -> booster_use -> box_complete
1,000b1cf405ea9ac1ae37ac761815cba7,start_level -> booster_use -> start_level
2,002d60067763bd3fb22ffd24ad1dd424,start_level -> booster_use -> box_complete -> ...
3,002f795faeb69377468c27da3364e743,start_level -> booster_use -> box_complete
4,008c00273819a7ccbcac538c1b687041,start_level -> booster_use -> box_complete
...,...,...
2223,ff8ea320a832ac7f4f52e98a2660bd7a,start_level -> booster_use -> box_complete -> ...
2224,ff934ad018e44b6ce2bfd79e7c109724,start_level -> booster_use -> box_complete -> ...
2225,ff9785c48aa0a3cb80f06202a5acdd0d,start_level -> booster_use -> box_complete -> ...
2226,ffd9c30e04c487cee4701bb35a060749,start_level -> booster_use -> booster_use


In [11]:
sequence_df['sequence'] = sequence_df['sequence'] + ' -> drop'

In [12]:
# Convert sequences to nodes and links for Sankey diagram
sequences = sequence_df['sequence'].str.split(' -> ')

# Get all unique events
all_events = set()
for seq in sequences:
    all_events.update(seq)

# Create nodes list
nodes = list(all_events)

# Create links dictionary to count transitions
links = {}
for seq in sequences:
    for i in range(len(seq)-1):
        source = seq[i]
        target = seq[i+1]
        key = (source, target)
        links[key] = links.get(key, 0) + 1

# Create source, target and value lists for plotting
source_list = []
target_list = []
value_list = []
for (source, target), value in links.items():
    source_list.append(nodes.index(source))
    target_list.append(nodes.index(target))
    value_list.append(value)

# Create Sankey diagram
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = nodes,
        color = "blue"
    ),
    link = dict(
        source = source_list,
        target = target_list,
        value = value_list
    )
)])

fig.update_layout(title_text="User Flow Sankey Diagram", font_size=10)
fig.show()

### Check user drop by inter level 8

In [124]:
# Get users whose sequence ends with 'INTER -> drop'
drop_after_inter_users = sequence_df[sequence_df['sequence'].str.endswith('INTER -> drop')]['user_pseudo_id'].tolist()
print(f"Number of users who dropped after INTER: {len(drop_after_inter_users)}")
drop_after_inter_users

Number of users who dropped after INTER: 681


['001a9c6f17af62e29f81d897699c7682',
 '002d60067763bd3fb22ffd24ad1dd424',
 '01016693579df9c72c3c0b94fcb9609c',
 '01a131f975505e3eec9ff4f499cd3826',
 '01c5688a99cd703075ac8847025d2ae3',
 '0218bde6e5ffbdae1262c1169769ddae',
 '02b5cac1fa7675b25be509f76f168e9a',
 '02b7cf8b02d5c23dea6914b37e7e335e',
 '03100d7a13c27e6388105157679d2de3',
 '03a4e0da0f6829938debaf649d1ae745',
 '03d55688ff013f044cc85116a659574a',
 '052935b77d0f2a9b4a547318082be01e',
 '055558027b1391ac279cb53d0daf6fc2',
 '06790555d13b8252757fb8da4a44db85',
 '068d98b7d0dd816ce04a4f381a261030',
 '06a6c4b79368cc74081ca10d2ec41c4d',
 '082b9cc05dc75c2fa289ba6cbe9f203a',
 '086ceded9631fe8e0566dbdc8b6551d7',
 '091c8bc0f4d264ee30782580c37227bf',
 '0963c8771d77baa98ad7dc424a3f133c',
 '09e98982f7be06dc169f5a264c7ffb7e',
 '0a9e582fbe671c32604284b024a6e273',
 '0b92a8f43943c8d1b2a270eee6db61c7',
 '0b97c6351a951f262a3b93f3645b23c4',
 '0cbfba9e8752a76a472239457787da4a',
 '0d01f9e7dfdce540d2e59e09da3b43cf',
 '0d77ff2a4b2ac502c5ec65f38609278e',
 

In [125]:
query = f"""
SELECT
  user_pseudo_id,
  event_name,
  event_timestamp,
  ev.key,
  COALESCE(ev.value.string_value,
           CAST(ev.value.int_value AS STRING),
           CAST(ev.value.float_value AS STRING),
           CAST(ev.value.double_value AS STRING)
  ) AS value
FROM
  `crazy-coffee-jam.analytics_483028617.events_intraday_*`,
  UNNEST(event_params) AS ev
WHERE
  _TABLE_SUFFIX BETWEEN '20250422' AND '20250519'
  and user_pseudo_id in ('{("','".join(drop_after_inter_users))}')
  and key not in ('firebase_event_origin','firebase_screen_class','ga_session_number', 'engaged_session_event', 'firebase_screen_id', 'ga_session_id', 'firebase_event_id',
  'system_app', 'previous_first_open_count', 'system_app_update', 'firebase_conversion', 'update_with_analytics', 'level_percent','IsLoop',
  'value', 'ad_platform', 'ad_unit_name', 'currency','ad_source')
ORDER BY
  user_pseudo_id,   
  event_timestamp
"""
sequence_user_drop_inter_8 = client.query(query).to_dataframe()

In [126]:
sequence_user_drop_inter_8

,user_pseudo_id,event_name,event_timestamp,key,value
0,001a9c6f17af62e29f81d897699c7682,first_open,1747259108906088,engagement_time_msec,1
1,001a9c6f17af62e29f81d897699c7682,screen_view,1747259109613089,entrances,1
2,001a9c6f17af62e29f81d897699c7682,firebase_campaign,1747259109983088,campaign_info_source,referrer API v2
3,001a9c6f17af62e29f81d897699c7682,firebase_campaign,1747259109983088,medium,(not set)
4,001a9c6f17af62e29f81d897699c7682,firebase_campaign,1747259109983088,source,(not set)
...,...,...,...,...,...
160388,fff7b258d6cddb69af658de0b448b768,screen_view,1747202541288265,firebase_previous_class,MessagingUnityPlayerActivity
160389,fff7b258d6cddb69af658de0b448b768,ad_impression,1747202542502252,ad_format,INTER
160390,fff7b258d6cddb69af658de0b448b768,ad_cgteam_impression,1747202542502267,ad_format,INTER
160391,fff7b258d6cddb69af658de0b448b768,user_engagement,1747202556799270,engagement_time_msec,15519


In [128]:
# Chỉ lấy các event sau khi user bắt đầu level 8
# Get the timestamp when each user started level 8
level_8_inter_times = sequence_user_drop_inter_8[
    (sequence_user_drop_inter_8['event_name'] == 'start_level') &
    (sequence_user_drop_inter_8['key'] == 'level') &
    (sequence_user_drop_inter_8['value'] == '8')
].groupby('user_pseudo_id')['event_timestamp'].min()

# Filter sequence_user to only include events after each user's level 8 start
sequence_user_drop_inter_8 = sequence_user_drop_inter_8.merge(
    level_8_start_times.reset_index(),
    on='user_pseudo_id',
    suffixes=('', '_level8_start')
)
sequence_user_drop_inter_8 = sequence_user_drop_inter_8[
    sequence_user_drop_inter_8['event_timestamp'] >= sequence_user_drop_inter_8['event_timestamp_level8_start']
].drop('event_timestamp_level8_start', axis=1)

# # Bỏ các dòng có event_name là booster_use và key là level và value là 8
# sequence_user = sequence_user[
#     ~((sequence_user['event_name'] == 'booster_use') &
#       (sequence_user['key'] == 'level') &
#       (sequence_user['value'] == '8'))
# ]

In [129]:
sequence_user_drop_inter_8

,user_pseudo_id,event_name,event_timestamp,key,value
128,001a9c6f17af62e29f81d897699c7682,start_level,1747259435318251,level,8
129,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,level,8
130,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_type,Frezze
131,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_method,Free
132,001a9c6f17af62e29f81d897699c7682,booster_use,1747259439858253,booster_type,Frezze
...,...,...,...,...,...
160388,fff7b258d6cddb69af658de0b448b768,screen_view,1747202541288265,firebase_previous_class,MessagingUnityPlayerActivity
160389,fff7b258d6cddb69af658de0b448b768,ad_impression,1747202542502252,ad_format,INTER
160390,fff7b258d6cddb69af658de0b448b768,ad_cgteam_impression,1747202542502267,ad_format,INTER
160391,fff7b258d6cddb69af658de0b448b768,user_engagement,1747202556799270,engagement_time_msec,15519


In [130]:
# prompt: sequence_user: Bỏ đi các dòng có key = level và value ứng với level đó != 8

sequence_user_drop_inter_8 = sequence_user_drop_inter_8[
    ~((sequence_user_drop_inter_8['key'] == 'level') &
      (pd.to_numeric(sequence_user_drop_inter_8['value'], errors='coerce') != 8))
]

In [131]:
# Drop rows where value is 'BANNER'
sequence_user_drop_inter_8 = sequence_user_drop_inter_8[sequence_user_drop_inter_8['value'] != 'BANNER']
sequence_user_drop_inter_8 = sequence_user_drop_inter_8[sequence_user_drop_inter_8['value'] != 'LEADER']
sequence_user_drop_inter_8 = sequence_user_drop_inter_8[sequence_user_drop_inter_8['value'] != 'REWARDED']

# For ad_impression events, replace event_name with the corresponding value
sequence_user_drop_inter_8.loc[sequence_user_drop_inter_8['event_name'] == 'ad_impression', 'event_name'] = sequence_user_drop_inter_8.loc[sequence_user_drop_inter_8['event_name'] == 'ad_impression', 'value']

sequence_user_drop_inter_8

,user_pseudo_id,event_name,event_timestamp,key,value
128,001a9c6f17af62e29f81d897699c7682,start_level,1747259435318251,level,8
129,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,level,8
130,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_type,Frezze
131,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_method,Free
132,001a9c6f17af62e29f81d897699c7682,booster_use,1747259439858253,booster_type,Frezze
...,...,...,...,...,...
160388,fff7b258d6cddb69af658de0b448b768,screen_view,1747202541288265,firebase_previous_class,MessagingUnityPlayerActivity
160389,fff7b258d6cddb69af658de0b448b768,INTER,1747202542502252,ad_format,INTER
160390,fff7b258d6cddb69af658de0b448b768,ad_cgteam_impression,1747202542502267,ad_format,INTER
160391,fff7b258d6cddb69af658de0b448b768,user_engagement,1747202556799270,engagement_time_msec,15519


In [132]:
sequence_user_drop_inter_8['key'].unique()

array(['level', 'booster_type', 'booster_method', 'box_id', 'Count',
       'resource_amount', 'resource_type', 'resource_gain_source',
       'firebase_previous_class', 'engagement_time_msec',
       'firebase_previous_id', 'ad_format', 'placement',
       'session_engaged', 'entrances', 'reward_type', 'rating',
       'error_message', 'revive_method', 'timestamp', 'fatal',
       'ad_unit_code', 'reward_value', 'previous_os_version',
       'campaign_info_source', 'gclid'], dtype=object)

In [145]:
sequence_user_drop_inter_8 = sequence_user_drop_inter_8[~sequence_user_drop_inter_8['key'].isin([
    'firebase_previous_class', 'firebase_previous_id', 'session_engaged', 'entrances', 
    'rating', 'reward_type', 'timestamp', 'fatal', 'ad_unit_code', 'previous_os_version',
    'campaign_info_source', 'gclid'
])]
sequence_user_drop_inter_8


,user_pseudo_id,event_name,event_timestamp,key,value
128,001a9c6f17af62e29f81d897699c7682,start_level,1747259435318251,level,8
129,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,level,8
130,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_type,Frezze
131,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_method,Free
132,001a9c6f17af62e29f81d897699c7682,booster_use,1747259439858253,booster_type,Frezze
...,...,...,...,...,...
160387,fff7b258d6cddb69af658de0b448b768,screen_view,1747202541288265,engagement_time_msec,251140
160389,fff7b258d6cddb69af658de0b448b768,INTER,1747202542502252,ad_format,INTER
160390,fff7b258d6cddb69af658de0b448b768,ad_cgteam_impression,1747202542502267,ad_format,INTER
160391,fff7b258d6cddb69af658de0b448b768,user_engagement,1747202556799270,engagement_time_msec,15519


In [149]:
# Group events by user_pseudo_id and aggregate event_names into a sequence
sequence_df_inter_8 = sequence_user_drop_inter_8.groupby('user_pseudo_id').agg(
    sequence=('event_name', lambda x: ' -> '.join(x))
).reset_index()

sequence_df_inter_8

,user_pseudo_id,sequence
0,001a9c6f17af62e29f81d897699c7682,start_level -> booster_gain -> booster_gain ->...
1,002d60067763bd3fb22ffd24ad1dd424,start_level -> booster_gain -> booster_gain ->...
2,01016693579df9c72c3c0b94fcb9609c,start_level -> booster_gain -> booster_gain ->...
3,01a131f975505e3eec9ff4f499cd3826,start_level -> booster_gain -> booster_gain ->...
4,01c5688a99cd703075ac8847025d2ae3,start_level -> booster_gain -> booster_gain ->...
...,...,...
676,ff73128446a78f62a5e088fd76236069,start_level -> booster_gain -> booster_gain ->...
677,ff78eeca154db2b41a8a39eacc9f4ad2,start_level -> booster_gain -> booster_gain ->...
678,ff8ea320a832ac7f4f52e98a2660bd7a,start_level -> booster_gain -> booster_gain ->...
679,ff9785c48aa0a3cb80f06202a5acdd0d,start_level -> booster_gain -> booster_gain ->...


In [150]:
# Split sequences by 'INTER ->' and keep only what comes after
sequence_df_inter_8['sequence'] = sequence_df_inter_8['sequence'].apply(lambda x: x.split('INTER -> ')[-1] if 'INTER -> ' in x else x)


In [152]:
# Remove ad_cgteam_impression from sequences
sequence_df_inter_8['sequence'] = sequence_df_inter_8['sequence'].apply(lambda x: x.replace('ad_cgteam_impression -> ', '').replace(' -> ad_cgteam_impression', ''))

sequence_df_inter_8

,user_pseudo_id,sequence
0,001a9c6f17af62e29f81d897699c7682,screen_view -> ads_inter_show -> watch_3_ads -...
1,002d60067763bd3fb22ffd24ad1dd424,user_engagement
2,01016693579df9c72c3c0b94fcb9609c,user_engagement
3,01a131f975505e3eec9ff4f499cd3826,user_engagement
4,01c5688a99cd703075ac8847025d2ae3,user_engagement
...,...,...
676,ff73128446a78f62a5e088fd76236069,user_engagement -> user_engagement
677,ff78eeca154db2b41a8a39eacc9f4ad2,user_engagement -> user_engagement
678,ff8ea320a832ac7f4f52e98a2660bd7a,user_engagement
679,ff9785c48aa0a3cb80f06202a5acdd0d,user_engagement


In [153]:
sequence_df_inter_8['sequence'] = sequence_df_inter_8['sequence'] + ' -> drop'

In [155]:
# Convert sequences to nodes and links for Sankey diagram
sequences = sequence_df_inter_8['sequence'].str.split(' -> ')

# Get all unique events
all_events = set()
for seq in sequences:
    all_events.update(seq)

# Create nodes list
nodes = list(all_events)

# Create links dictionary to count transitions
links = {}
for seq in sequences:
    for i in range(len(seq)-1):
        source = seq[i]
        target = seq[i+1]
        key = (source, target)
        links[key] = links.get(key, 0) + 1

# Create source, target and value lists for plotting
source_list = []
target_list = []
value_list = []
for (source, target), value in links.items():
    source_list.append(nodes.index(source))
    target_list.append(nodes.index(target))
    value_list.append(value)

# Create Sankey diagram
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = nodes,
        color = "blue"
    ),
    link = dict(
        source = source_list,
        target = target_list,
        value = value_list
    )
)])

fig.update_layout(title_text="User Flow Sankey Diagram", font_size=10)
fig.show()

In [156]:
sequence_user_drop_inter_8

,user_pseudo_id,event_name,event_timestamp,key,value
128,001a9c6f17af62e29f81d897699c7682,start_level,1747259435318251,level,8
129,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,level,8
130,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_type,Frezze
131,001a9c6f17af62e29f81d897699c7682,booster_gain,1747259437823252,booster_method,Free
132,001a9c6f17af62e29f81d897699c7682,booster_use,1747259439858253,booster_type,Frezze
...,...,...,...,...,...
160387,fff7b258d6cddb69af658de0b448b768,screen_view,1747202541288265,engagement_time_msec,251140
160389,fff7b258d6cddb69af658de0b448b768,INTER,1747202542502252,ad_format,INTER
160390,fff7b258d6cddb69af658de0b448b768,ad_cgteam_impression,1747202542502267,ad_format,INTER
160391,fff7b258d6cddb69af658de0b448b768,user_engagement,1747202556799270,engagement_time_msec,15519


In [158]:
sequence_user_drop_inter_8 = sequence_user_drop_inter_8[
    (sequence_user_drop_inter_8['event_name'] == 'user_engagement') & 
    (sequence_user_drop_inter_8['key'] == 'engagement_time_msec')
]
sequence_user_drop_inter_8

,user_pseudo_id,event_name,event_timestamp,key,value
187,001a9c6f17af62e29f81d897699c7682,user_engagement,1747259559748293,engagement_time_msec,2518
190,001a9c6f17af62e29f81d897699c7682,user_engagement,1747510612346612,engagement_time_msec,23114
406,002d60067763bd3fb22ffd24ad1dd424,user_engagement,1747346734538751,engagement_time_msec,53865
646,01016693579df9c72c3c0b94fcb9609c,user_engagement,1747072534025811,engagement_time_msec,4606
882,01a131f975505e3eec9ff4f499cd3826,user_engagement,1746648700899764,engagement_time_msec,2791
...,...,...,...,...,...
159704,ff78eeca154db2b41a8a39eacc9f4ad2,user_engagement,1746420590716352,engagement_time_msec,5748
159916,ff8ea320a832ac7f4f52e98a2660bd7a,user_engagement,1746294260769313,engagement_time_msec,2553
160126,ff9785c48aa0a3cb80f06202a5acdd0d,user_engagement,1747480730682761,engagement_time_msec,1860
160391,fff7b258d6cddb69af658de0b448b768,user_engagement,1747202556799270,engagement_time_msec,15519


In [ ]:
sequence_user_drop_inter_8['value_second'] = sequence_user_drop_inter_8['value']/1000
sequence_user_drop_inter_8

TypeError: cannot convert the series to <class 'int'>

# Sequence of user drop level 5

In [2]:
query = """
with a as (
select distinct user_pseudo_id
from `crazy-coffee-jam.flatten_table.start_level`
where level >= 6
and user_pseudo_id in (select distinct user_pseudo_id from `crazy-coffee-jam.flatten_table.first_open` where event_date >= '2025-05-01')
)
select distinct user_pseudo_id
from `crazy-coffee-jam.flatten_table.start_level`
where event_date >= '2025-05-01'
and level = 5
and user_pseudo_id not in (select user_pseudo_id from a)
and user_pseudo_id in (select distinct user_pseudo_id from `crazy-coffee-jam.flatten_table.first_open` where event_date >= '2025-05-01')
"""
user_drop_level_5 = client.query(query).to_dataframe()
user_drop_level_5_list = user_drop_level_5['user_pseudo_id'].tolist()

In [3]:
query = f"""
SELECT
  user_pseudo_id,
  event_name,
  event_timestamp,
  ev.key,
  COALESCE(ev.value.string_value,
           CAST(ev.value.int_value AS STRING),
           CAST(ev.value.float_value AS STRING),
           CAST(ev.value.double_value AS STRING)
  ) AS value
FROM
  `crazy-coffee-jam.analytics_483028617.events_intraday_*`,
  UNNEST(event_params) AS ev
WHERE
  _TABLE_SUFFIX BETWEEN '20250422' AND '20250519'
  and user_pseudo_id in ('{("','".join(user_drop_level_5_list))}')
  and event_name in ('first_open','start_level', 'win_level', 'lose_level', 'revive_level', 'booster_use', 'box_complete','ad_impression')
  and key in ('level','ad_format')
ORDER BY
  user_pseudo_id,
  event_timestamp
"""
sequence_user = client.query(query).to_dataframe()

In [4]:
# Chỉ lấy các event sau khi user bắt đầu level 5
# Get the timestamp when each user started level 5
level_5_start_times = sequence_user[
    (sequence_user['event_name'] == 'start_level') &
    (sequence_user['key'] == 'level') &
    (sequence_user['value'] == '5')
].groupby('user_pseudo_id')['event_timestamp'].min()

# Filter sequence_user to only include events after each user's level 5 start
sequence_user = sequence_user.merge(
    level_5_start_times.reset_index(),
    on='user_pseudo_id',
    suffixes=('', '_level5_start')
)
sequence_user = sequence_user[
    sequence_user['event_timestamp'] >= sequence_user['event_timestamp_level5_start']
].drop('event_timestamp_level5_start', axis=1)

In [88]:
# Count number of rows for each level value where key = 'level'
level_counts = sequence_user[sequence_user['key'] == 'level'].groupby('value').size().reset_index(name='count')
print("Level counts:")
print(level_counts)

Level counts:
  value  count
0     2      2
1     5  14821
2     6     29
3     7     13
4     8     11


In [5]:
sequence_user[(sequence_user['key'] == 'level') & (sequence_user['value'] != '5')]

,user_pseudo_id,event_name,event_timestamp,key,value
434,0153160ff01b0a54f4c2b4f783ff5a72,box_complete,1747416283741951,level,6
435,0153160ff01b0a54f4c2b4f783ff5a72,box_complete,1747416286921952,level,6
436,0153160ff01b0a54f4c2b4f783ff5a72,box_complete,1747416290850953,level,6
438,0153160ff01b0a54f4c2b4f783ff5a72,box_complete,1747416295141956,level,6
439,0153160ff01b0a54f4c2b4f783ff5a72,box_complete,1747416298346957,level,6
440,0153160ff01b0a54f4c2b4f783ff5a72,box_complete,1747416300604960,level,6
441,0153160ff01b0a54f4c2b4f783ff5a72,win_level,1747416302619961,level,6
16169,3d83dad85726392450694fb9ae7a456c,box_complete,1747288797547417,level,7
17815,444de5b74ba41c4b611f15e3a848bd18,box_complete,1746468799223841,level,8
17816,444de5b74ba41c4b611f15e3a848bd18,box_complete,1746468802990842,level,8


In [6]:
# prompt: sequence_user: Bỏ đi các dòng có key = level và value ứng với level đó != 5

sequence_user = sequence_user[
    ~((sequence_user['key'] == 'level') &
      (pd.to_numeric(sequence_user['value'], errors='coerce') != 5))
]

In [7]:
# Count number of rows for each level value where key = 'level'
level_counts = sequence_user[sequence_user['key'] == 'level'].groupby('value').size().reset_index(name='count')
print("Level counts:")
print(level_counts)

Level counts:
  value  count
0     5  13529


In [8]:
# Drop rows where value is 'BANNER'
sequence_user = sequence_user[sequence_user['value'] != 'BANNER']
sequence_user = sequence_user[sequence_user['value'] != 'LEADER']
sequence_user = sequence_user[sequence_user['value'] != 'REWARDED']

# For ad_impression events, replace event_name with the corresponding value
sequence_user.loc[sequence_user['event_name'] == 'ad_impression', 'event_name'] = sequence_user.loc[sequence_user['event_name'] == 'ad_impression', 'value']

sequence_user

,user_pseudo_id,event_name,event_timestamp,key,value
23,00223c8a066cf5def277d0bd1b624520,start_level,1747671229624689,level,5
47,002d927079c92234e66ccbe6b762766a,start_level,1746996672881689,level,5
48,002d927079c92234e66ccbe6b762766a,box_complete,1746996681983663,level,5
49,002d927079c92234e66ccbe6b762766a,box_complete,1746996689369664,level,5
51,002d927079c92234e66ccbe6b762766a,box_complete,1746996694428669,level,5
...,...,...,...,...,...
65859,ffc35e2ce3d07baa3e94f57b65660b94,box_complete,1747036223238551,level,5
65860,ffc35e2ce3d07baa3e94f57b65660b94,box_complete,1747036227208552,level,5
65861,ffc35e2ce3d07baa3e94f57b65660b94,box_complete,1747036232216555,level,5
65863,ffc35e2ce3d07baa3e94f57b65660b94,win_level,1747036235674561,level,5


In [9]:
# Group events by user_pseudo_id and aggregate event_names into a sequence
sequence_df = sequence_user.groupby('user_pseudo_id').agg(
    sequence=('event_name', lambda x: ' -> '.join(x))
).reset_index()

sequence_df

,user_pseudo_id,sequence
0,00223c8a066cf5def277d0bd1b624520,start_level
1,002d927079c92234e66ccbe6b762766a,start_level -> box_complete -> box_complete ->...
2,00346aeac6385c1f33dcb30324a1eb57,start_level -> box_complete -> box_complete ->...
3,004bdd18a49e46be8c9e17f7f34bbbc0,start_level -> box_complete -> box_complete ->...
4,006b14bd697814ab9a03cd604f9d0cb5,start_level -> box_complete -> box_complete ->...
...,...,...
1948,fefbe4b50b3c7f3b454311aee6b20cea,start_level -> box_complete -> box_complete ->...
1949,ff17a9ce28e030117a4a3e98876c19ad,start_level -> box_complete -> box_complete ->...
1950,ff1f79f4517241f3f318ea6c63659c7d,start_level -> box_complete -> box_complete ->...
1951,ff7f3f23864360f7727a6576d619e76b,start_level -> box_complete -> box_complete ->...


In [10]:
# Replace multiple consecutive box_complete events with a single box_complete
sequence_df['sequence'] = sequence_df['sequence'].str.replace(r'(box_complete(?:\s*->\s*box_complete)*)', 'box_complete', regex=True)
sequence_df

,user_pseudo_id,sequence
0,00223c8a066cf5def277d0bd1b624520,start_level
1,002d927079c92234e66ccbe6b762766a,start_level -> box_complete -> win_level -> INTER
2,00346aeac6385c1f33dcb30324a1eb57,start_level -> box_complete -> win_level -> INTER
3,004bdd18a49e46be8c9e17f7f34bbbc0,start_level -> box_complete
4,006b14bd697814ab9a03cd604f9d0cb5,start_level -> box_complete -> win_level -> INTER
...,...,...
1948,fefbe4b50b3c7f3b454311aee6b20cea,start_level -> box_complete -> win_level -> INTER
1949,ff17a9ce28e030117a4a3e98876c19ad,start_level -> box_complete -> win_level -> INTER
1950,ff1f79f4517241f3f318ea6c63659c7d,start_level -> box_complete -> win_level -> INTER
1951,ff7f3f23864360f7727a6576d619e76b,start_level -> box_complete -> win_level -> INTER


In [11]:
sequence_df['sequence'] = sequence_df['sequence'] + ' -> drop'

In [97]:
# Convert sequences to nodes and links for Sankey diagram
sequences = sequence_df['sequence'].str.split(' -> ')

# Get all unique events
all_events = set()
for seq in sequences:
    all_events.update(seq)

# Create nodes list
nodes = list(all_events)

# Create links dictionary to count transitions
links = {}
for seq in sequences:
    for i in range(len(seq)-1):
        source = seq[i]
        target = seq[i+1]
        key = (source, target)
        links[key] = links.get(key, 0) + 1

# Create source, target and value lists for plotting
source_list = []
target_list = []
value_list = []
for (source, target), value in links.items():
    source_list.append(nodes.index(source))
    target_list.append(nodes.index(target))
    value_list.append(value)

# Create Sankey diagram
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = nodes,
        color = "blue"
    ),
    link = dict(
        source = source_list,
        target = target_list,
        value = value_list
    )
)])

fig.update_layout(title_text="User Flow Sankey Diagram", font_size=10)
fig.show()

In [12]:
# Get users whose sequence ends with 'INTER -> drop'
drop_after_inter_users = sequence_df[sequence_df['sequence'].str.endswith('INTER -> drop')]['user_pseudo_id'].tolist()
print(f"Number of users who dropped after INTER: {len(drop_after_inter_users)}")
drop_after_inter_users

Number of users who dropped after INTER: 1083


['002d927079c92234e66ccbe6b762766a',
 '00346aeac6385c1f33dcb30324a1eb57',
 '006b14bd697814ab9a03cd604f9d0cb5',
 '00b60bd696941f44a8831ecb24bf606c',
 '013bfb15f5ef405542cba988c9a95c60',
 '0153160ff01b0a54f4c2b4f783ff5a72',
 '01d76ffab575ee94e4ce9177f54985cb',
 '01e486fb54a62851722f9a40bfcece37',
 '025fd95c4e0b1db47a1fce028c586073',
 '0268eab389e13aab015ebf168eb2f726',
 '026ee9873e904bc1943102173d90e2cf',
 '0288a1fddf09e1c3352a33c278418117',
 '02e27b7ce173b1281f5f021e099806b6',
 '035f724e7fcd61cd41869eddf4844c6c',
 '0377470c2b8b39eecff8620687d42de7',
 '03927decbba2c85ba5b35814e5533ffe',
 '03ba186e5860454a09747bac7f7508c8',
 '03c6b77e671c23b2a85bffaf6c074d7a',
 '03d9fdbbd40f68a7a4160f1ca1dcbf79',
 '040bb4039388a9635027d3d45937829f',
 '045f76972d4d5a27d51331aa7ff105e3',
 '048d5620af05769f487d685ccd90ec6c',
 '04ac92f156d42176541ad022c7bb0944',
 '051dc2e4b707d0e1152afeb92f2b63d2',
 '058b51f34fdb0cbcf4556947941a64f0',
 '05c37d944fb1f57e4590472eace4294e',
 '05e2986b8f1d142b1e9e379ec91e979b',
 

In [ ]:
query  = """ 
select distinct user_pseudo_id 
from `crazy-coffee-jam.flatten_table.af_inter_complete`
where level = 5 
and user_pseudo_id in ('{("','".join(drop_after_inter_users))}')
"""
finish_inter_level_5_drop = client.query(query).to_dataframe()
len(finish_inter_level_5_drop)

15816

In [16]:
len(drop_after_inter_users)

1083

# Sequence of user drop level 15

In [98]:
query = """
with a as (
select distinct user_pseudo_id
from `crazy-coffee-jam.flatten_table.start_level`
where level >= 16
and user_pseudo_id in (select distinct user_pseudo_id from `crazy-coffee-jam.flatten_table.first_open` where event_date >= '2025-05-01')
)
select distinct user_pseudo_id
from `crazy-coffee-jam.flatten_table.start_level`
where event_date >= '2025-05-01'
and level = 15
and user_pseudo_id not in (select user_pseudo_id from a)
and user_pseudo_id in (select distinct user_pseudo_id from `crazy-coffee-jam.flatten_table.first_open` where event_date >= '2025-05-01')
"""
user_drop_level_15 = client.query(query).to_dataframe()
user_drop_level_15_list = user_drop_level_15['user_pseudo_id'].tolist()

In [100]:
query = f"""
SELECT
  user_pseudo_id,
  event_name,
  event_timestamp,
  ev.key,
  COALESCE(ev.value.string_value,
           CAST(ev.value.int_value AS STRING),
           CAST(ev.value.float_value AS STRING),
           CAST(ev.value.double_value AS STRING)
  ) AS value
FROM
  `crazy-coffee-jam.analytics_483028617.events_intraday_*`,
  UNNEST(event_params) AS ev
WHERE
  _TABLE_SUFFIX BETWEEN '20250422' AND '20250522'
  and user_pseudo_id in ('{("','".join(user_drop_level_15_list))}')
  and event_name in ('start_level', 'win_level', 'lose_level', 'revive_level', 'booster_use', 'box_complete','ad_impression')
  and key in ('level', 'ad_format' )
ORDER BY
  user_pseudo_id,
  event_timestamp
"""
sequence_user = client.query(query).to_dataframe()

In [101]:
# Chỉ lấy các event sau khi user bắt đầu level 15
# Get the timestamp when each user started level 15
level_15_start_times = sequence_user[
    (sequence_user['event_name'] == 'start_level') &
    (sequence_user['key'] == 'level') &
    (sequence_user['value'] == '15')
].groupby('user_pseudo_id')['event_timestamp'].min()

# Filter sequence_user to only include events after each user's level 15 start
sequence_user = sequence_user.merge(
    level_15_start_times.reset_index(),
    on='user_pseudo_id',
    suffixes=('', '_level15_start')
)
sequence_user = sequence_user[
    sequence_user['event_timestamp'] >= sequence_user['event_timestamp_level15_start']
].drop('event_timestamp_level15_start', axis=1)

In [102]:
# prompt: sequence_user: Bỏ đi các dòng có key = level và value ứng với level đó != 15

sequence_user = sequence_user[
    ~((sequence_user['key'] == 'level') &
      (pd.to_numeric(sequence_user['value'], errors='coerce') != 15))
]

In [103]:
# Count number of rows for each level value where key = 'level'
level_counts = sequence_user[sequence_user['key'] == 'level'].groupby('value').size().reset_index(name='count')
print("Level counts:")
print(level_counts)

Level counts:
  value  count
0    15  21252


In [104]:
# Drop rows where value is 'BANNER'
sequence_user = sequence_user[sequence_user['value'] != 'BANNER']
sequence_user = sequence_user[sequence_user['value'] != 'LEADER']

In [105]:
# For ad_impression events, replace event_name with the corresponding value
sequence_user.loc[sequence_user['event_name'] == 'ad_impression', 'event_name'] = sequence_user.loc[sequence_user['event_name'] == 'ad_impression', 'value']

sequence_user

,user_pseudo_id,event_name,event_timestamp,key,value
142,001ab56758356022472c8733566cbaec,start_level,1747576899721078,level,15
143,001ab56758356022472c8733566cbaec,box_complete,1747576917350055,level,15
144,001ab56758356022472c8733566cbaec,box_complete,1747576959034056,level,15
145,001ab56758356022472c8733566cbaec,box_complete,1747577061078057,level,15
146,001ab56758356022472c8733566cbaec,box_complete,1747577072682058,level,15
...,...,...,...,...,...
291786,fffeda73def58ca84be21651950e6f9f,box_complete,1747271876386610,level,15
291791,fffeda73def58ca84be21651950e6f9f,start_level,1747348037338954,level,15
291793,fffeda73def58ca84be21651950e6f9f,box_complete,1747348051099269,level,15
291796,fffeda73def58ca84be21651950e6f9f,booster_use,1747348075001106,level,15


In [106]:
# Group events by user_pseudo_id and aggregate event_names into a sequence
sequence_df = sequence_user.groupby('user_pseudo_id').agg(
    sequence=('event_name', lambda x: ' -> '.join(x))
).reset_index()

sequence_df

,user_pseudo_id,sequence
0,001ab56758356022472c8733566cbaec,start_level -> box_complete -> box_complete ->...
1,001daff6aff341f8c689b4031fafae51,start_level -> box_complete -> box_complete ->...
2,001fe2fb86263dcd8980cfbe84bdd308,start_level -> box_complete -> box_complete ->...
3,0028ffde246d3660ccb6e48ece54d639,start_level -> box_complete -> box_complete ->...
4,003547177934245a5cae775495e538ba,start_level -> box_complete -> box_complete ->...
...,...,...
1150,ffa125f4882c2ba9d9ac531dee4399c4,start_level -> box_complete -> box_complete ->...
1151,ffa89ce0643ce653e45bc2fcb3491fd4,start_level -> box_complete -> box_complete ->...
1152,ffa9cf7bb08c4ddaa2814cae437bbee9,start_level -> box_complete -> box_complete ->...
1153,ffbd3ad282ec49d5081370f18d346e95,start_level -> box_complete -> box_complete ->...


In [107]:
# Replace multiple consecutive box_complete events with a single box_complete
sequence_df['sequence'] = sequence_df['sequence'].str.replace(r'(box_complete(?:\s*->\s*box_complete)*)', 'box_complete', regex=True)
sequence_df

,user_pseudo_id,sequence
0,001ab56758356022472c8733566cbaec,start_level -> box_complete -> lose_level -> s...
1,001daff6aff341f8c689b4031fafae51,start_level -> box_complete -> lose_level -> I...
2,001fe2fb86263dcd8980cfbe84bdd308,start_level -> box_complete -> lose_level -> I...
3,0028ffde246d3660ccb6e48ece54d639,start_level -> box_complete
4,003547177934245a5cae775495e538ba,start_level -> box_complete -> lose_level -> I...
...,...,...
1150,ffa125f4882c2ba9d9ac531dee4399c4,start_level -> box_complete -> REWARDED -> sta...
1151,ffa89ce0643ce653e45bc2fcb3491fd4,start_level -> box_complete -> REWARDED -> REW...
1152,ffa9cf7bb08c4ddaa2814cae437bbee9,start_level -> box_complete -> lose_level
1153,ffbd3ad282ec49d5081370f18d346e95,start_level -> box_complete


In [108]:
sequence_df['sequence'] = sequence_df['sequence'] + ' -> drop'

In [109]:
# Convert sequences to nodes and links for Sankey diagram
sequences = sequence_df['sequence'].str.split(' -> ')

# Get all unique events
all_events = set()
for seq in sequences:
    all_events.update(seq)

# Create nodes list
nodes = list(all_events)

# Create links dictionary to count transitions
links = {}
for seq in sequences:
    for i in range(len(seq)-1):
        source = seq[i]
        target = seq[i+1]
        key = (source, target)
        links[key] = links.get(key, 0) + 1

# Create source, target and value lists for plotting
source_list = []
target_list = []
value_list = []
for (source, target), value in links.items():
    source_list.append(nodes.index(source))
    target_list.append(nodes.index(target))
    value_list.append(value)

# Create Sankey diagram
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = nodes,
        color = "blue"
    ),
    link = dict(
        source = source_list,
        target = target_list,
        value = value_list
    )
)])

fig.update_layout(title_text="User Flow Sankey Diagram", font_size=10)
fig.show()